In [14]:
from scipy import stats
from scipy.stats import norm, ttest_ind
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Задание 1
Сколько несовместных независимых экспериментов можно запустить одновременно, если за время эксперимента собираем 10000 наблюдений?

Если решаем запустить 10 экспериментов, то на каждый эксперимент можно выделить по 1000 наблюдений, размер групп будет равен 500.

Параметры экспериментов:

- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.


Надо посчитать количество экспериментов, чтобы в каждом были эффекты сверху. То есть надо посчитать количество наблюдений для одного эксперимента с учетом поправки на множественное тестирование и разделить 10к на это число

In [15]:
def get_sample_size_abs(epsilon, std, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * (2 * std ** 2) / (epsilon ** 2)
        )
    )
    return sample_size

In [16]:
def get_sample_size_arb(mu, std, eff=1.01, alpha=0.05, beta=0.2):
    epsilon = (eff - 1) * mu

    return get_sample_size_abs(epsilon, std=std, alpha=alpha, beta=beta)

In [17]:
total_observation = 10000
alpha = 0.05
beta = 0.1
effect = 1.03
mean = 100
std = 10

total_observation / (get_sample_size_arb(mean, std, effect, alpha=alpha, beta=beta) * 2)

21.367521367521366

In [18]:
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm

# параметры эксперимента
total_size = 10000
mean_ = 100
std_ = 10
effect = 0.03
alpha = 0.05
beta = 0.1


def estimate_sample_size(effect, std, alpha, beta):
    """Оценка необходимого размер групп."""
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    var = 2 * std ** 2
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    return sample_size


# оценим необходимый размер групп
sample_size = estimate_sample_size(effect * 100, 10, alpha, beta)
print(f'sample_size = {sample_size}')
# вычислим количество экспериментов
count_exp = total_size / (sample_size * 2)
print(f'count_exp = {count_exp:0.1f}')


def estimate_ci_bernoulli(p, n, alpha=0.05):
    """Доверительный интервал для Бернуллиевской случайной величины."""
    t = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    std_n = np.sqrt(p * (1 - p) / n)
    return p - t * std_n, p + t * std_n

# Проверим, что при 21 эксперимента ошибки контролируются на заданных уровнях, а при 22 экспериментах нет.

for count_exp in [20, 21, 22]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in range(10000):
        a, b = np.random.normal(mean_, std_, (2, sample_size,))
        b_effect = b * (1 + effect)
        errors_aa.append(stats.ttest_ind(a, b).pvalue < alpha)
        errors_ab.append(stats.ttest_ind(a, b_effect).pvalue >= alpha)

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')


sample_size = 233
count_exp = 21.5
count_exp = 20
sample_size = 250
оценка вероятности ошибки I рода = 0.0546
  доверительный интервал = [0.0501, 0.0591]
оценка вероятности ошибки II рода = 0.0945
  доверительный интервал = [0.0888, 0.1002]
count_exp = 21
sample_size = 238
оценка вероятности ошибки I рода = 0.0504
  доверительный интервал = [0.0461, 0.0547]
оценка вероятности ошибки II рода = 0.1078
  доверительный интервал = [0.1017, 0.1139]
count_exp = 22
sample_size = 227
оценка вероятности ошибки I рода = 0.0477
  доверительный интервал = [0.0435, 0.0519]
оценка вероятности ошибки II рода = 0.1211
  доверительный интервал = [0.1147, 0.1275]


# Задание 2

Задача похожа на предыдущую, только теперь решение принимается не независимо для каждого эксперимента.
Например, у нас есть 5 текстов для маркетинговой рассылки, хотим проверить, какой эффективнее работает и работает ли вообще.

Алгоритм будет следующий:

1. Формируем непересекающиеся контрольные и экспериментальные группы для каждого из 5 вариантов.
2. Проводим параллельно 5 экспериментов.
3. С помощью метода Холма определяем, в каких экспериментах были статистически значимые отличия.
4. Если значимых отличий не обнаружено, то говорим, что эффекта нет, все варианты отклоняем.
5. Если значимые отличия обнаружены, то из вариантов со значимым эффектом выбираем вариант с наименьшим значением p-value, будем использовать его.

Будем считать, что совершена ошибка I рода, если найдены значимые отличия, когда на самом деле их не было ни в одном из вариантов.

Будем считать, что совершена ошибка II рода, если:
- либо не найдено значимых отличий, когда на самом деле они были;
- либо выбранный для дальнейшего использования вариант на самом деле был без эффекта, при этом были варианты с эффектом.

Параметры экспериментов:

- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Замечание: при оценке вероятности ошибки II рода нужно рассматривать худший сценарий, когда эффект есть только в одном из экспериментов. Чем в большем количестве экспериментов будет эффект, тем меньше будет вероятность ошибки II рода.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.

In [20]:
def method_holm(pvalues, alpha=0.05):
    """Применяет метод Холма для проверки значимости изменений.
    
    pvalues - List[float] - список pvalue.
    alpha - float, уровень значимости.
    return - np.array, массив из нулей и единиц, 0 - эффекта нет, 1 - эффект есть.
    """
    m = len(pvalues)
    array_alpha = np.arange(m, 0, -1)
    array_alpha = alpha / array_alpha
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue < alpha_:
            res[pvalue_index] = 1
        else:
            break
    res = res.astype(int)
    return res

# Проверим, что при 12 эксперимента ошибки контролируются на заданных уровнях, а при 13 экспериментах нет.

for count_exp in [12, 13]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in (range(10000)):
        list_ab_values = [
            np.random.normal(mean_, std_, (2, sample_size))
            for _ in range(count_exp)
        ]
        # синтетический А/А тест
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        aa_with_effect = method_holm(pvalues, alpha)
        errors_aa.append(np.sum(aa_with_effect) > 0)

        # Синтетический А/Б тест.
        # Достаточно проверить случай, когда эффект есть лишь в одном из экспериментов,
        # так как при наличии эффектов в большем кол-ве экспериментов ошибок II рода станет меньше.
        # Добавим эффект в первый эксперимент (не важно в какой добавлять, так как данные случайные)
        list_ab_values[0][1] *= 1 + effect
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        ab_with_effect = method_holm(pvalues, alpha)
        if np.sum(ab_with_effect) == 0:
            # если эффектов не найдено, то это ошибка
            errors_ab.append(True)
        else:
            # если эффектов найден где его нет, то это ошибка
            errors_ab.append(np.min(pvalues) != pvalues[0])

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')

count_exp = 12
sample_size = 416
оценка вероятности ошибки I рода = 0.0491
  доверительный интервал = [0.0449, 0.0533]
оценка вероятности ошибки II рода = 0.0870
  доверительный интервал = [0.0815, 0.0925]
count_exp = 13
sample_size = 384
оценка вероятности ошибки I рода = 0.0489
  доверительный интервал = [0.0447, 0.0531]
оценка вероятности ошибки II рода = 0.1252
  доверительный интервал = [0.1187, 0.1317]


In [27]:
set(sum([[1,2], [1,2], [1,3], [1,3]], []))

{1, 2, 3}